# Topic Model
In this part we run a topic model analysis 

In [1]:
!python -m pip install json-lines

In [2]:
import json_lines

with open('Illinois-20200302-xml/data/data.jsonl/data.jsonl', 'rb') as f: # opening file in binary(rb) mode 
    df = list(json_lines.reader(f))
    for item in json_lines.reader(f):
        print(item) #or use print(item['X']) for printing specific data
        break

In [3]:
import json

tot = len(df)
json_list = []

for n in range(0,tot):
    json_list.append(json.dumps(df[n]['casebody']))


In [4]:
import re
import nltk
import json
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt')

def clean_text(text):
    # Convert words to lower case
    text = text.lower() 

    text = re.sub('<.*?>', ' ', text)
    text = re.sub('[a-zA-Z]+\\\\[a-zA-Z]+', ' ', text)
    text = re.sub('\\\\[a-zA-Z]+', ' ', text)
    text = re.sub('\\\\"', '"', text)
    text = re.sub('[0-9]+[a-zA-Z]+', '', text)
    text = re.sub('\*', '', text)
    text = re.sub('amp', '', text)
    text = re.sub('[0-9]+', '', text)
    
    #Remove stopwords
    stop_words = set(stopwords.words('english'))
    text = " ".join([i for i in text.split() if i not in stop_words])
    
    #Remove punctuation
    words = nltk.word_tokenize(text)
    text = [word for word in words if word.isalnum()]

    return text

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\tompe\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\tompe\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [5]:
# for topic model

texts = []

for sentence in json_list:
    texts.append(clean_text(sentence))

In [6]:
# Necessary because pyLDAvis make some trouble

from IPython.display import HTML
css_str = '<style> \
.jp-icon-warn0 path {fill: var(--jp-warn-color0);} \
.bp3-button-text path { fill: var(--jp-inverse-layout-color3);} \
.jp-icon-brand0 path { fill: var(--jp-brand-color0);} \
text.terms { fill: #616161;} \
</style>'
display(HTML(css_str))

### Topic model preparation

In [7]:
# Set training parameters.
num_topics = 30
chunksize = 2000
passes = 35
iterations = 600
eval_every = None  # Don't evaluate model perplexity, takes too much time

In [8]:
# Remove rare and common tokens.
import gensim
from gensim.models import Word2Vec
from gensim.corpora import Dictionary
import gensim.corpora as corpora


In [9]:
# Create a dictionary representation of the documents.
id2word = corpora.Dictionary(texts)

corpus = [id2word.doc2bow(text) for text in texts]


## LDA

In [10]:

# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=num_topics, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=chunksize,
                                           passes=passes,
                                           alpha='auto',
                                           per_word_topics=True,
                                           eval_every=eval_every,
                                           iterations=iterations)

In [11]:
import re
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

# Print the Keyword in the 10 topics
#pprint(lda_model.print_topics())
#doc_lda = lda_model[corpus]

lda_topics = lda_model.get_topics()

for topicno in range(num_topics):
    print('Topic {}'.format(topicno))
    print([(x, round(y, 2)) for x, y in lda_model.show_topic(topicno, topn=10)], '\n')

Topic 0
[('contract', 0.05), ('agreement', 0.03), ('parties', 0.01), ('would', 0.01), ('made', 0.01), ('time', 0.01), ('terms', 0.01), ('agreed', 0.01), ('written', 0.01), ('upon', 0.01)] 

Topic 1
[('trust', 0.07), ('estate', 0.06), ('trustee', 0.03), ('court', 0.02), ('property', 0.02), ('administrator', 0.02), ('probate', 0.02), ('real', 0.01), ('bank', 0.01), ('interest', 0.01)] 

Topic 2
[('said', 0.09), ('bill', 0.04), ('decree', 0.03), ('court', 0.02), ('upon', 0.02), ('complainant', 0.02), ('made', 0.01), ('defendants', 0.01), ('equity', 0.01), ('filed', 0.01)] 

Topic 3
[('death', 0.04), ('estate', 0.02), ('deceased', 0.02), ('wife', 0.02), ('husband', 0.01), ('will', 0.01), ('children', 0.01), ('testator', 0.01), ('life', 0.01), ('said', 0.01)] 

Topic 4
[('railroad', 0.03), ('company', 0.02), ('street', 0.02), ('car', 0.02), ('negligence', 0.01), ('train', 0.01), ('track', 0.01), ('appellee', 0.01), ('cars', 0.01), ('appellant', 0.01)] 

Topic 5
[('testified', 0.02), ('time'

In [12]:
tid = id2word.token2id['court']
lda_model.get_term_topics(tid)

[(1, 0.018587256),
 (2, 0.01985069),
 (6, 0.015566355),
 (9, 0.0554741),
 (11, 0.030019019),
 (12, 0.023092253),
 (16, 0.02031378),
 (22, 0.022921333),
 (23, 0.068013616),
 (24, 0.012154705),
 (25, 0.052616578),
 (29, 0.014162113)]

In [13]:
!pip install pyLDAvis

In [14]:
# Plotting tools
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
pyLDAvis.enable_notebook()

import matplotlib.pyplot as plt
%matplotlib inline

# Visualize the topics
pyLDAvis.enable_notebook()
vis = gensimvis.prepare(lda_model, corpus, id2word)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
12     0.172815  0.035078       1        1  6.894370
16    -0.014127 -0.076501       2        1  6.833962
5      0.031538 -0.140092       3        1  5.806279
29     0.122015  0.055681       4        1  5.679004
18     0.161923 -0.062686       5        1  5.647851
6      0.136425  0.147409       6        1  5.102002
9      0.027276  0.187227       7        1  4.819446
17    -0.092903 -0.062010       8        1  4.727907
22     0.108613  0.098619       9        1  4.078034
0     -0.114556 -0.069517      10        1  3.966223
21    -0.171745 -0.082653      11        1  3.653188
25    -0.004674  0.082828      12        1  3.607610
2     -0.177065  0.030140      13        1  3.031598
14    -0.047388 -0.113847      14        1  2.931383
19     0.079569 -0.183695      15        1  2.834999
23    -0.048537  0.271257      16        1  2.771597
24    -0.088349  0.023251      17        1  2.705925
8     -0.013832 -0.129486      18        1  2.612991
20    -0.084077  0.043263      19        1  2.489043
11     0.118782  0.085091      20        1  2.405150
26     0.102642  0.061258      21        1  2.385347
28    -0.214473 -0.007252      22        1  2.185632
4      0.041022 -0.183855      23        1  2.044228
10     0.173675 -0.087344      24        1  1.920157
3     -0.141814 -0.060852      25        1  1.800939
7      0.188625 -0.098325      26        1  1.576702
13     0.010354  0.083904      27        1  1.454128
15    -0.041812 -0.009062      28        1  1.432215
27    -0.082818  0.082172      29        1  1.357744
1     -0.137105  0.079999      30        1  1.244345, topic_info=           Term          Freq         Total Category  logprob  loglift
1385  defendant  1.942926e+06  1.942926e+06  Default  30.0000  30.0000
816   plaintiff  1.023600e+06  1.023600e+06  Default  29.0000  29.0000
981        said  9.501330e+05  9.501330e+05  Default  28.0000  28.0000
278       court  2.845296e+06  2.845296e+06  Default  27.0000  27.0000
63    appellant  4.759380e+05  4.759380e+05  Default  26.0000  26.0000
...         ...           ...           ...      ...      ...      ...
827       power  1.151121e+04  1.490017e+05  Topic30  -5.2971   1.8259
197       claim  1.090848e+04  2.770563e+05  Topic30  -5.3509   1.1519
452       filed  1.200247e+04  5.596057e+05  Topic30  -5.2553   0.5444
762       order  1.122197e+04  4.864579e+05  Topic30  -5.3225   0.6173
275      county  1.112559e+04  4.977723e+05  Topic30  -5.3311   0.5857

[2537 rows x 6 columns], token_table=       Topic      Freq    Term
term                          
0          1  0.067178       a
0          4  0.574033       a
0          5  0.028168       a
0          6  0.041626       a
0          7  0.051733       a
...      ...       ...     ...
5546      18  0.992284  zoning
5546      21  0.007696  zoning
16380      6  0.150414  zurich
16380     20  0.005988  zurich
16380     28  0.842951  zurich

[11020 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[13, 17, 6, 30, 19, 7, 10, 18, 23, 1, 22, 26, 3, 15, 20, 24, 25, 9, 21, 12, 27, 29, 5, 11, 4, 8, 14, 16, 28, 2])

In [15]:
# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=texts, dictionary=id2word, coherence='u_mass') #c_v
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)

C:\Users\tompe\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)



Coherence Score:  -1.0193124596850156


## Latent Semantic Indexin

In [59]:
from gensim.models import LsiModel

model = LsiModel(corpus, id2word=id2word, num_topics=num_topics)

vectors = model[corpus]

type(vectors)

gensim.interfaces.TransformedCorpus

In [60]:
topics = model.get_topics()
topics.shape

(30, 386575)

In [61]:

for topicno in range(num_topics):
    print('Topic {}'.format(topicno))
    print([(x, round(y, 2)) for x, y in model.show_topic(topicno, topn=10)], '\n')

Topic 0
[('court', 0.44), ('defendant', 0.4), ('trial', 0.22), ('evidence', 0.16), ('case', 0.15), ('would', 0.13), ('plaintiff', 0.13), ('said', 0.12), ('state', 0.12), ('people', 0.11)] 

Topic 1
[('defendant', 0.66), ('said', -0.21), ('trial', 0.16), ('people', 0.15), ('court', -0.14), ('upon', -0.13), ('property', -0.13), ('act', -0.12), ('plaintiff', -0.11), ('company', -0.1)] 

Topic 2
[('said', -0.52), ('court', 0.37), ('defendant', -0.25), ('trial', 0.2), ('company', -0.15), ('upon', -0.14), ('app', 0.14), ('section', 0.14), ('made', -0.12), ('motion', 0.11)] 

Topic 3
[('plaintiff', -0.74), ('court', 0.19), ('state', 0.18), ('people', 0.15), ('evidence', -0.14), ('section', 0.11), ('said', 0.11), ('county', 0.11), ('act', 0.1), ('jury', -0.1)] 

Topic 4
[('court', -0.42), ('act', 0.31), ('section', 0.24), ('trial', -0.22), ('state', 0.2), ('said', -0.19), ('city', 0.17), ('board', 0.16), ('public', 0.15), ('illinois', 0.12)] 

Topic 5
[('defendant', 0.46), ('evidence', -0.38),

In [62]:
coherence = CoherenceModel(model=model,
                              corpus=corpus,
                              dictionary=id2word,
                              coherence='u_mass')
print(coherence.get_coherence())

-1.16028219230003


In [30]:
tid = id2word.token2id['court']
lda_model.get_term_topics(tid)

[(2, 0.02158861),
 (5, 0.022726068),
 (6, 0.06882597),
 (9, 0.019694839),
 (11, 0.022138435),
 (12, 0.014439493),
 (16, 0.010350409),
 (20, 0.029297294),
 (22, 0.03261238),
 (23, 0.050701145),
 (24, 0.0158358),
 (28, 0.016544117),
 (29, 0.03536669)]

In [55]:
narcotics = ['alcohol','narcotics','narco','narcotic','cannabis', 'cocaine', 'methamphetamine', 'smart drugs', 'marijuana', 'MDMA', 'LSD', 'KETAMINA', 'heroin', 'fentanyl','drugs']
weapons = ['weapons','weapon','gun', 'knife', 'weapon', 'firearm', 'rifle', 'carabine', 'shotgun', 'assaults rifle', 'sword', 'blunt objects','pistol'] 
investigation = ['investigation','police','detective','gang', 'crime','robber', 'black','mafia', 'serial kiler', 'rape', 'thefts', 'recidivism', 'arrest', 'ethnicity', 'caucasian', 'afroamerican', 'native american', 'hispanic', 'gender', 'male', 'female', 'man', 'woman', 'girl', 'boy', 'robbery', 'cybercrime']

In [56]:
for n in narcotics:
    try:
        tid = id2word.token2id[n]
        print(n + ":")
        print(lda_model.get_term_topics(n))
    except:
        print(n + " not find")

alcohol:
[(4, 0.019034302)]
narcotics:
[]
narco:
[]
narcotic:
[]
cannabis:
[]
cocaine:
[]
methamphetamine not find
smart drugs not find
marijuana:
[]
MDMA not find
LSD not find
KETAMINA not find
heroin:
[]
fentanyl:
[]
drugs:
[]


In [57]:
for w in weapons:
    try:
        tid = id2word.token2id[w]
        print(w + ":")
        print(lda_model.get_term_topics(tid))
    except:
        print(w + " not find")

weapons:
[]
weapon:
[]
gun:
[]
knife:
[]
weapon:
[]
firearm:
[]
rifle:
[]
carabine not find
shotgun:
[]
assaults rifle not find
sword:
[]
blunt objects not find
pistol:
[]


In [58]:
for i in investigation:
    try:
        tid = id2word.token2id[i]
        print(i + ":")
        print(lda_model.get_term_topics(tid))
    except:
        print(i + " not find")

investigation:
[]
police:
[(18, 0.017122451)]
detective:
[]
gang:
[]
crime:
[(16, 0.010470388)]
robber:
[]
black:
[]
mafia:
[]
serial kiler not find
rape:
[]
thefts:
[]
recidivism:
[]
arrest:
[]
ethnicity:
[]
caucasian:
[]
afroamerican not find
native american not find
hispanic:
[]
gender:
[]
male:
[]
female:
[]
man:
[]
woman:
[]
girl:
[]
boy:
[]
robbery:
[]
cybercrime not find
